<a id='top'></a>

# Watershed and Catchment Landcover Metrics

## Aspect and Percent North Covariates
Calculate aspect grid with the DEM used to create the flow network for each processing area (e.g. HUC8 in NHDPlus or
 region for BB, PWS, or Kodiak). We need the DEMs used for each flow direction grid, or we can't calculate covariates at
 the watershed scale.
 * **~~aspect_rch = calculate mean aspect for stream reach (zonal statistics)~~**
 * **~~aspect_cat = calculate mean aspect over catchments (zonal statistics)~~**
 * **north_wtd = create 1/0 grid of north facing cells from aspect grid (north = aspects from 315-45 degrees), ~~~use
  weighted flow accumulation to sum north facing cells, divide by flow accumulation grid to get % of north facing
  cells in each watershed~~~ Use tabulate area to quantify area of North cells (VALUE_1) and non-North (VALUE_0) cells
  for each watershed and calculate percent north as north_wtd = row[VALUE_1]/(row[VALUE_1]+row[VALUE_0])*100 .**
## Elevation Metrics
Calculate elevation metrics for catchment/watershed with temperature data using zonal statistics as table.
* <b> Running "ALL" zonal statistics as it does not allow you to choose two types of statistics
(ex statistics_type = ["MIN","MAX"]) instead set statistics_type="ALL"</b>
### Catchment Elevation Metrics
* **cat_elev_mn = mean elevation for catchment**
* **cat_elev_min = minimum elevation for catchment**
* **cat_elev_max = max elevation for catchment**
* **cat_elev_std = standard deviation of elevation for catchment**
### Watershed Elevation Metrics
* **wtd_elev_mn = mean watershed elevation**
* **wtd_elev_min = min watershed elevation**
* **wtd_elev_max = max watershed elevation**
* **wtd_elev_sd (or cv) = standard deviation of watershed elevation**
## Slope Metrics
Calculate slope metrics for catchment/watershed with temperature data using zonal statistics as table and
statistics_type = "ALL".
### Catchment Slope Metrics
* **cat_slope_mn = mean slope for catchment**
* **cat_slope_min = minimum slope for catchment**
* **cat_slope_max = max slope for catchment**
* **cat_slope_std = standard deviation of slope for catchment**
### Watershed Elevation Metrics
* **wtd_slope_mn = mean watershed slope**
* **wtd_slope_min = min watershed slope**
* **wtd_slope_max = max watershed slope**
* **wtd_slope_sd (or cv) = standard deviation of watershed slope**
## Lake, Wetland and Glacier Cover
Lake/Pond type waterbodies from NHDPLus hydrography for those regions with NHDPlus derived datasets and NHD waterbodies
for those regions with TauDEM derived waterbodies.  Merge all waterbodies together (FTYPE = 390) and use tabulate area
calculate percent cover of lakes in catchments/watersheds.
 * **Created local copies of NHDPlus and NHD waterbodies and exported to T driver here <>  **
Use NLCD grid (wetlands.tif) and tabulate area on wetlands grid with watersheds (sum) / divide by number of cells in
watershed from fac grid, wetlands from NLCD
## Distance from coast
DO NOT WORK ON THIS


## Import modules

In [3]:
import arcpy
import os
import datetime
import time
import sys
arcpy.env.overwriteOutput = True
today = datetime.datetime.now()
# Make the time stamp.
time_stamp = '{:%d%m%Y}'.format(today)
print(time_stamp)

path = os.getcwd()
print (path)
print (sys.base_exec_prefix)

03122021
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\landcover
C:\Users\dwmerrigan\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3


## Create working output location to store intermediate data
Create folder and gdb to store output data.
Set data_dir to folder containing all AKSSF regional subfolders/geoadatabases
* RS data folder: data_dir = r"W:\\GIS\\AKSSF"
* DM data folder: data_dir = r"D:\\GIS\\AKSSF"

In [4]:
# Set AKSSF Data directory
# data_dir = r"W:\GIS\AKSSF"

data_dir = r"D:\\GIS\\AKSSF"
arcpy.env.workspace = data_dir
regions = arcpy.ListWorkspaces(workspace_type="Folder")
rois= []
print (regions)

# Path to create output folder/gdb
temppath = r"D:\\GIS\\" # Output folder
dirname = 'AKSSF_land_met'
tempgdbname = 'AKSSF_land_met.gdb'
temp_dir = os.path.join(temppath, dirname)

# Create temporary working gdb
if not arcpy.Exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print('Working Folder already created...', temp_dir)

outcheck = os.path.join(temp_dir, tempgdbname)

if arcpy.Exists(outcheck):
    print ('Output location already exists', outcheck)
    outgdb = outcheck
if not arcpy.Exists(outcheck):
    print('Creating output GDB')
    tempgdb = arcpy.CreateFileGDB_management(temp_dir,tempgdbname)
    print ('Output geodatabase created at', outcheck)
    outgdb = tempgdb.getOutput(0)

['D:\\\\GIS\\\\AKSSF\\Bristol_Bay', 'D:\\\\GIS\\\\AKSSF\\Cook_Inlet', 'D:\\\\GIS\\\\AKSSF\\Copper_River', 'D:\\\\GIS\\\\AKSSF\\Kodiak', 'D:\\\\GIS\\\\AKSSF\\Prince_William_Sound']
Working Folder already created... D:\\GIS\\AKSSF_land_met
Output location already exists D:\\GIS\\AKSSF_land_met\AKSSF_land_met.gdb


In [5]:
# Set regional workspaces from AKSSF data folder and store in list
arcpy.env.workspace = data_dir
# Use All of the cores on the machine
arcpy.env.parallelProcessingFactor = "100%"

regions  = arcpy.ListWorkspaces(workspace_type="Folder")
regions

['D:\\\\GIS\\\\AKSSF\\Bristol_Bay',
 'D:\\\\GIS\\\\AKSSF\\Cook_Inlet',
 'D:\\\\GIS\\\\AKSSF\\Copper_River',
 'D:\\\\GIS\\\\AKSSF\\Kodiak',
 'D:\\\\GIS\\\\AKSSF\\Prince_William_Sound']

## Create aspect and north grids if they do not already exists
### NHDPlus

In [4]:
# Code to create merged elevation grid, aspect and north aspect grids for entire AKSSF region with NHDPlus elev data

# elrasters = []
# nhd_dat = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\AKSSF_NHDPlus\\extracts"
# for dirpath, dirname, filenames in arcpy.da.Walk(nhd_dat, 'RasterDataset'):
#     for filename in filenames:
#         if filename == "elev_cm.tif":
#             elras = os.path.join(dirpath, filename)
#             elrasters.append(elras)
#             print (f'Appending {elras} to list')
#
# spatial_ref = arcpy.Describe(elrasters[0]).spatialReference
# print (spatial_ref.name)
# # Merge all nhdplus elevation rasters together
# elevname = 'AKSSF_NHDPlus_elev_cm.tif'
# AKSSF_elev_cm = arcpy.MosaicToNewRaster_management(elrasters,temp_dir,elevname, spatial_ref, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")
#
# #make aspect raster for all of akssf from merged nhdplus
# from arcpy.sa import *
# aspname = 'AKSSF_NHDPlus_aspect.tif'
# asp_rast = Aspect(in_raster= AKSSF_elev_cm, method='Planar')
# asp_rast.save(os.path.join(temp_dir, aspname))
#
# # make north raster from aspect raster
# norname = 'AKSSF_NHDPlus_north_aspect.tif'
# aspect_in = Raster(asp_rast)
# north_raster = Con((aspect_in>=0)&(aspect_in<=45),1,Con((aspect_in<=360)&(aspect_in>=315),1,0))
# north_raster.save(os.path.join(temp_dir, norname))

### TauDEM - Create grids using Timms Composite DEM for entire TauDEM processed areas

In [5]:
# Code to create merged elevation grid, aspect and north aspect grids for entire AKSSF region with TauDEM elev data
# compDEM = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\data\topography\AKSSF_Elevation_Composite_10m.tif"
# clip to study area
# AKSSF_elev_10m = r"D:\\GIS_temp\\AKSSF_Composite_10m_extract.tif"
#
# #make aspect raster for all of akssf from merged nhdplus
# from arcpy.sa import *
# aspname = 'AKSSF_Composite_10m_aspect.tif'
# asp_rast = Aspect(in_raster= AKSSF_elev_10m, method='Planar')
# asp_rast.save(os.path.join(temp_dir, aspname))
#
# # make north raster from aspect raster
# norname = 'AKSSF_Composite_10m_north_aspect.tif'
# aspect_in = Raster(asp_rast)
# north_raster = Con((aspect_in>=0)&(aspect_in<=45),1,Con((aspect_in<=360)&(aspect_in>=315),1,0))
# north_raster.save(os.path.join(temp_dir, norname))

## Begin GIS portion
- May be faster and save more space to create aspect rasters here using extract by mask?
- May be able to use aspect raster with tabulate intersection to calculate percent North (aspects from 315-45 degrees)

### <b>UPDATE - 2021-10-12 Do not run aspect metrics at this time. Only need to run percent North for watersheds</b>

In [6]:
from arcpy.sa import *
import re

# Set data_dir equal to folder containing AKSSF regional subfolders containing GDBs and raster datasets
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338) #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
regions  = arcpy.ListWorkspaces(workspace_type="Folder")

# Lists for variables not needed at present time
#cat_asp_ztables = []
#wtd_asp_ztables = []
#cat_pernorth_taba_tables=[]

# Lists to store output tables
wtd_pernorth_taba_tables=[]
wtd_lp_tabint_tables = []
wtd_glac_tabint_tables = []
wtd_wet_taba_tables = []
cat_elev_ztables = []
wtd_elev_ztables = []
cat_slope_ztables = []
wtd_slope_ztables = []


# Clear lists
cat_cur_fields = []
wtd_cur_fields = []

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

# Seperate data by
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Loop through all processing areas
rois = nhdplus_dat + tauDem_dat

for roi in rois:
    # Loop through regional folders
    for region in regions:
        if roi in str(region):
            print(f'{roi} using data from {region} folder')
            # Set data and variables unique to regions with NHDPlus Data
            if roi in nhdplus_dat:
                lakes_fc = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHDPlus_LakePond_alb"
                # Fields for update cursor
                cat_cur_fields = ['cat_ID_txt', 'NHDPlusID',"cat_ID_con"]
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
                print (f'{roi} in {nhdplus_dat} AKSSF list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
                print('----------')
            # Set data and variables unique to regions with TauDEM Data
            elif roi in tauDem_dat:
                lakes_fc = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHD_LakesPonds_alb"
                # Fields for update cursor
                if roi == 'Bristol_Bay':
                    cat_cur_fields = ['cat_ID_txt', 'catID',"cat_ID_con"]
                    wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
                else:
                    cat_cur_fields = ['cat_ID_txt', 'gridcode',"cat_ID_con"]
                    wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
                print (f'{roi} in {tauDem_dat} TauDEM list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
                print('----------')
            wtds = []
            cats = []
            # Start iter timing function
            iteration_start = time.time()
            # Set workspace to region folder
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            walk = arcpy.da.Walk(region, datatype = ['FeatureClass','RasterDataset'])
            for dirpath, dirnames, filenames in walk:
                for filename in filenames:
                    # Create list of watersheds
                    if (("wtd" in filename) and ("merge" not in filename)):
                        #print(f" {filename} watershed identified")
                        wtds.append(os.path.join(dirpath, filename))
                    # Set merged watersheds dataset
                    elif 'wtds_merge'== filename:
                        wtdpath = os.path.join(dirpath,filename)
                        wtdname = roi +'_'+filename
                        # Make local copy projected in AKAlbers
                        wtd_merge = arcpy.FeatureClassToFeatureClass_conversion(wtdpath,outgdb,wtdname)
                        print(f'Merged watershed dataset {filename} found')
                        print('----------')
                        wtdfieldnames = []
                        wtdlstFields = arcpy.ListFields(wtd_merge)
                        for field in wtdlstFields:
                            wtdfieldnames.append(field.name)
                        if str(wtd_cur_fields[0]) in wtdfieldnames:
                            print (f'{wtd_cur_fields[0]} field already in dataset')
                            print('----------')
                        else:
                            print (f'Adding {wtd_cur_fields[0]} field to watershed dataset {wtd_merge}')
                            print('----------')
                            # add cat_ID_txt field and concat cat_ID + region
                            arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[0]),field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields[0:2]) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[0] = strval.replace('.0',"")
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)
                        if str(wtd_cur_fields[2]) in wtdfieldnames:
                            print (f'{wtd_cur_fields[2]} field already in dataset {wtd_merge}')
                            print('----------')
                        else:
                            print (f'Adding {wtd_cur_fields[2]} field to watershed dataset {wtd_merge}')
                            print('----------')
                            # add cat_ID_con field and concat cat_ID + region
                            arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[2]),field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[2] = str(roi) +'_'+ strval.replace(".0","")
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)

                    # Select glaciers fc
                    elif 'glaciers' in filename:
                        # Make local copy projected in AKAlbers
                        glacpath = os.path.join(dirpath, filename)
                        glacname = roi+'_'+filename
                        glac_fc = arcpy.FeatureClassToFeatureClass_conversion(glacpath,outgdb,glacname)

                    # Select elevation raster
                    elif 'elev.tif' == filename:
                        elev_rast = os.path.join(dirpath, filename)

                    # # Select aspect raster
                    # elif 'aspect' in filename:
                    #     asp_rast = os.path.join(dirpath, filename)

                    # Select north raster
                    elif 'north.tif' == filename:
                        nor_rast = os.path.join(dirpath, filename)

                    # Select slope raster
                    elif 'slope.tif' == filename:
                        slope_rast = os.path.join(dirpath, filename)

                    # Select wetland raster
                    elif 'wetlands.tif' == filename:
                        wet_rast = os.path.join(dirpath, filename)

                    # Select catch_int fc (catchments of interest for region) and make a copy
                    elif 'cats_intersect' in filename:
                        # Make local copy projected in AKAlbers
                        catspath = os.path.join(dirpath,filename)
                        catsname = roi+"_"+filename
                        cats = arcpy.FeatureClassToFeatureClass_conversion(catspath, outgdb,catsname)
                        catlstfields = arcpy.ListFields(cats)
                        catfieldnames = []
                        for field in catlstfields:
                            catfieldnames.append(field.name)
                        if str(cat_cur_fields[0]) in catfieldnames:
                            print (f'{cat_cur_fields[0]} field already in dataset {cats}')
                            print('----------')
                        else:
                            print (f'Adding {cat_cur_fields[0]} field to catchment dataset {cats}')
                            print('----------')
                            # add cat_ID_txt field
                            arcpy.AddField_management(cats, str(cat_cur_fields[0]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(cats, cat_cur_fields[0:2]) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[0] = strval.replace('.0',"")
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)
                        if str(cat_cur_fields[2]) in catfieldnames:
                            print (f'{cat_cur_fields[2]} field already in dataset {cats}')
                            print('----------')
                        else:
                            print (f'Adding {cat_cur_fields[2]} field to catchment dataset {cats}')
                            print('----------')
                            # add cat_ID_txt field & cat_ID + region concat field
                            arcpy.AddField_management(cats,str(cat_cur_fields[2]),field_type='TEXT')
                            # populate cat_ID_con
                            with arcpy.da.UpdateCursor(cats, cat_cur_fields) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[2] = str(roi) +'_'+ strval.replace('.0',"")
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)

            print (f'Calculating topographic metrics for catchments & watersheds of interest in {roi}'
                   f' region')
            print ('----------')
            print(f'Geodatabase: {gdb}')
            print ('----------')
            print (f'Watershed Merge: {wtd_merge}')
            print (f'  Projection {arcpy.Describe(wtd_merge).spatialReference.name}')
            print ('----------')
            print (f'Catchment Intersect: {cats}')
            print (f'  Projection {arcpy.Describe(cats).spatialReference.name}')
            print ('----------')
            print (f'Elevation Raster: {elev_rast}')
            print (f'  Projection: {arcpy.Describe(elev_rast).spatialReference.name}')
            print ('----------')
            print (f'North Aspect Raster: {nor_rast}')
            print (f'  Projection: {arcpy.Describe(nor_rast).spatialReference.name}')
            print ('----------')
            print (f'Wetlands Raster: {wet_rast}')
            print (f'  Projection {arcpy.Describe(wet_rast).spatialReference.name}')
            print ('----------')
            print (f'Slope Raster: {slope_rast}')
            print (f'  Projection {arcpy.Describe(slope_rast).spatialReference.name}')
            print ('----------')        
            print (f'Lakes Ponds fc: {lakes_fc}')
            print (f'  Projection {arcpy.Describe(lakes_fc).spatialReference.name}')
            print ('----------')
            print (f'Glaciers fc: {glac_fc} ')
            print (f'  Projection {arcpy.Describe(glac_fc).spatialReference.name}')
            print ('----------')
            print (f'{len(wtds)} Watersheds to process')
            print ('----------')
            print (f'Catchment intersect {cats} selected')
            print ('----------')

            try:
                # Set output names/paths for watershed and catchment tables
                wtd_merge_name = roi + "_Watersheds_Merge"
                wtd_merge_path = os.path.join(outgdb,wtd_merge_name)

                # # Aspect variables
                # wtd_merge_asp_table_name = roi + "_Watersheds_Merge_AspectZstats"
                # wtd_merge_asp_table_path = os.path.join(outgdb, wtd_merge_asp_table_name)
                # cat_asp_table_name = roi + "_Catchments_AspectZstats"
                # cat_asp_table_path = os.path.join(outgdb, cat_asp_table_name)

                # Percent North variables
                wtd_merge_pernorth_table_name = roi + "_Watersheds_Merge_PercentNorth"
                wtd_merge_pernorth_table_path = os.path.join(outgdb, wtd_merge_pernorth_table_name)
                # cat_pernorth_table_name = roi + "_Catchments_PercentNorth"
                # cat_pernorth_table_path = os.path.join(outgdb, cat_pernorth_table_name)

                # Elevation variables
                wtd_merge_elev_table_name = roi + "_Watersheds_Merge_ElevZstats"
                wtd_merge_elev_table_path = os.path.join(outgdb, wtd_merge_elev_table_name)
                cat_elev_table_name = roi + "_Catchments_ElevZstats"
                cat_elev_table_path = os.path.join(outgdb, cat_elev_table_name)
                
                # Slope variables
                wtd_merge_slope_table_name = roi + "_Watershed_Merge_SlopeZstats"
                wtd_merge_slope_table_path = os.path.join(outgdb, wtd_merge_slope_table_name)
                cat_slope_table_name = roi + "_Catchments_SlopeZstats"
                cat_slope_table_path = os.path.join(outgdb, cat_slope_table_name)

                # Lakes Ponds variables
                wtd_merge_lp_table_name = roi + "_Watershed_Merge_LakesPonds"
                wtd_merge_lp_table_path = os.path.join(outgdb, wtd_merge_lp_table_name)
                cat_lp_table_name = roi + "_Catchments_LakesPonds"
                cat_lp_path = os.path.join(outgdb, cat_lp_table_name)

                # Wetlands variables
                wtd_merge_wetlands_table_name = roi + "_Watershed_Merge_Wetlands"
                wtd_merge_wetlands_table_path = os.path.join(outgdb, wtd_merge_wetlands_table_name)
                cat_wetlands_table_name = roi + "_Catchments_Wetlands"
                cat_wetlands_table_path = os.path.join(outgdb, cat_wetlands_table_name)

                # Glaciers
                wtd_merge_glac_table_name = roi + "_Watershed_Merge_Glaciers"
                wtd_merge_glac_table_path = os.path.join(outgdb, wtd_merge_glac_table_name)
                cat_glac_table_name = roi + "_Catchments_Glaciers"
                cat_glac_table_path = os.path.join(outgdb, cat_glac_table_name)

                # # Watersheds already merged so no need to run at this time
                # if not arcpy.Exists(wtd_merge):
                #     mergestart = time.time()
                #     # Merge watersheds
                #     wtd_merge = arcpy.Merge_management( wtds, wtd_merge_path ,add_source='ADD_SOURCE_INFO')
                #     # Add wtd_id field
                #     arcpy.AddField_management(wtd_merge,'cat_ID_txt',field_type='TEXT')
                #     arcpy.AddField_management(wtd_merge,"cat_ID_con",field_type='TEXT')
                #     # Add region field
                #     arcpy.AddField_management(wtd_merge,'region',field_type='TEXT')
                #     # Populate watershed id and region information using update cursor - faster than field calc
                #     with arcpy.da.UpdateCursor(wtd_merge,['MERGE_SRC','cat_ID_txt','region','cat_ID_con']) as cur:
                #         for row in cur:
                #             row[1] = re.findall('\d+', row[0])[0]
                #             row[2] = roi
                #             row[3] = + str(roi) + '_'+ str(re.findall('\d+', row[0])[0])
                #             # Update
                #             cur.updateRow(row)
                #         del(row)
                #     del(cur)
                #     mergestop = time.time()
                #     mergetime = int (mergestop - mergestart)
                #     print(f'Watershed Merge for {roi} Elapsed time: ({datetime.timedelta(seconds=mergetime)})')
                #     print('----------')
                # else:
                #     print (f'Merged watershed dataset {wtd_merge} already created')
                #     print('----------')

                # Begin Zonal Stats
                zstat_start = time.time()
                print(f'Begin Elevation zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
                      f' region')

                # Elevation Zonal statistics  for watersheds
                print(f'Calculating {roi} watershed elevation zonal stats...')
                arcpy.env.snapRaster = elev_rast
                arcpy.env.cellSize = elev_rast
                wtd_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                                zone_field = wtd_cur_fields[0],
                                                                in_value_raster = elev_rast,
                                                                out_table = wtd_merge_elev_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for watershed tables                                                )
                arcpy.AddField_management(wtd_elev_metrics_table,'region',field_type='TEXT')
                # Add cat_ID_Con field
                arcpy.AddField_management(wtd_elev_metrics_table,'cat_ID_con',field_type='TEXT')
                # Update region field
                with arcpy.da.UpdateCursor(wtd_elev_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                    for row in cur:
                        row[0] = roi
                        strval = str(row[1])
                        row[2] = roi+"_"+strval.replace(".0","")
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed elev table to list
                wtd_elev_ztables.append(wtd_elev_metrics_table)

                # Elevation zonal statistics for catchments
                print(f'Calculating {roi} catchment elevation zonal stats...')
                arcpy.env.snapRaster = elev_rast
                arcpy.env.cellSize = elev_rast
                cat_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                                zone_field = cat_cur_fields[0],
                                                                in_value_raster = elev_rast,
                                                                out_table = cat_elev_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for catchment table
                arcpy.AddField_management(cat_elev_metrics_table,'region',field_type='TEXT')
                # Add cat_ID_Con field
                arcpy.AddField_management(cat_elev_metrics_table,'cat_ID_con',field_type='TEXT')

                # Update fields
                with arcpy.da.UpdateCursor(cat_elev_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                    for row in cur:
                        row[0] = roi
                        strval = str(row[1])
                        row[2] = roi+"_"+strval.replace(".0","")
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append catchment elev table to list
                cat_elev_ztables.append(cat_elev_metrics_table)

                # Slope zonal statistics for catchments
                print(f'Calculating {roi} catchment slope zonal stats...')
                arcpy.env.snapRaster = slope_rast
                arcpy.env.cellSize = slope_rast
                cat_slope_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                                zone_field = cat_cur_fields[0],
                                                                in_value_raster = slope_rast,
                                                                out_table = cat_slope_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for catchment table
                arcpy.AddField_management(cat_slope_metrics_table,'region',field_type='TEXT')
                # Add cat_ID_Con field
                arcpy.AddField_management(cat_slope_metrics_table,'cat_ID_con',field_type='TEXT')

                # Update region field
                with arcpy.da.UpdateCursor(cat_slope_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                    for row in cur:
                        row[0] = roi
                        strval =str(row[1])
                        row[2] = roi+"_"+strval.replace(".0","")
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append catchment slope table to list
                cat_slope_ztables.append(cat_slope_metrics_table)

                # Watershed slope Zonal Statistics
                print(f'Begin Slope zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
                      f' region')

                # Elevation Zonal statistics  for watersheds
                print(f'Calculating {roi} watershed slope zonal stats...')
                arcpy.env.snapRaster = slope_rast
                arcpy.env.cellSize = slope_rast
                wtd_slope_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                                zone_field = wtd_cur_fields[0],
                                                                in_value_raster = slope_rast,
                                                                out_table = wtd_merge_slope_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for watershed tables                                                )
                arcpy.AddField_management(wtd_slope_metrics_table,'region',field_type='TEXT')
                # Add cat_ID_Con field
                arcpy.AddField_management(wtd_slope_metrics_table,'cat_ID_con',field_type='TEXT')

                # Update region field
                with arcpy.da.UpdateCursor(wtd_slope_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                    for row in cur:
                        row[0] = roi
                        strval = str(row[1])
                        row[2] = roi+"_"+strval.replace(".0","")
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed slope table to list
                wtd_slope_ztables.append(wtd_slope_metrics_table)

                # # Aspect Zonal statistics  for watersheds
                # print(f'Calculating {roi} watershed aspect zonal stats...')
                # wtd_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge, zone_field ="cat_ID_txt",
                #                                                in_value_raster = asp_rast, out_table = wtd_merge_asp_table_path,
                #                                                statistics_type='ALL')
                # arcpy.AddField_management(wtd_asp_metrics_table, 'region', field_type='TEXT')
                # Add cat_ID_Con field
                # arcpy.AddField_management(wtd_asp_metrics_table,'cat_ID_con',field_type='TEXT')
                # arcpy.CalculateField_management(wtd_asp_metrics_table, 'region', 'roi')
                # Update region field
                # with arcpy.da.UpdateCursor(wtd_asp_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                #     for row in cur:
                #         row[0] = roi
                #         strval = str(row[1])
                #         row[2] = roi+"_"+strval.replace(".0","")
                #         # Update
                #         cur.updateRow(row)
                #     del(row)
                # del(cur)
                # wtd_asp_ztables.append(wtd_asp_metrics_table)

                # # Aspect Zonal statistics for catchments
                # print(f'Calculating {roi} catchment aspect zonal stats...')
                # cat_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats, zone_field ="cat_ID_txt",
                #                                                in_value_raster = asp_rast, out_table = cat_asp_table_path,
                #                                                statistics_type='ALL')
                # arcpy.AddField_management(cat_asp_metrics_table, 'region', field_type='TEXT')
                # Add cat_ID_Con field
                # arcpy.AddField_management(cat_asp_metrics_table,'cat_ID_con',field_type='TEXT')
                # arcpy.CalculateField_management(cat_asp_metrics_table, 'region', 'roi')
                # Update region field
                # with arcpy.da.UpdateCursor(cat_asp_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                #     for row in cur:
                #         strval = str(row[1])
                #         row[2] = roi+"_"+strval.replace(".0","")
                #         # Update
                #         cur.updateRow(row)
                #     del(row)
                # del(cur)
                # cat_asp_ztables.append(cat_asp_metrics_table)

                zstat_stop = time.time()
                zstat_time = int (zstat_stop - zstat_start)
                print(f'Zonal Stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')
                print('----------')

                # Tabulate Area with north grid and watersheds
                tabarea_start = time.time()
                print(f'Begin tabulate area of north facing cells for watersheds and catchments in {roi} region')
                print('----------')
                # Percent North Tabulate area for watersheds
                wtd_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                              zone_field= wtd_cur_fields[0],
                                                              in_class_data=nor_rast,
                                                              class_field="Value",
                                                              out_table = wtd_merge_pernorth_table_path
                                                              )
                # Add region and percent north fields
                arcpy.AlterField_management(wtd_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
                arcpy.AddField_management(wtd_per_north_tabarea, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_per_north_tabarea, 'wtd_north_per', field_type='Float')
                arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[0], field_type='TEXT')
                arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[2], field_type='TEXT')
                wtdnorfields = [f.name for f in arcpy.ListFields(wtd_per_north_tabarea)]
                print (wtdnorfields)
                with arcpy.da.UpdateCursor(wtd_per_north_tabarea, wtdnorfields) as cur:
                    for row in cur:
                        strval = str(row[1])
                        row[4] = roi
                        row[5] = row[3]/(row[3]+row[2])*100
                        row[6] = strval.replace('.0','')
                        row[7] = roi +'_'+ strval.replace(".0","")
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Drop UPPERCASE field form tab area
                arcpy.DeleteField_management(wtd_per_north_tabarea,'CAT_ID_TXT_DEL')
                # Append watershed percent north table to list
                wtd_pernorth_taba_tables.append(wtd_per_north_tabarea)

                # Percent Lakes Ponds using Tabulate Intersection for watersheds
                print(f'Begin tabulate intersection between {lakes_fc} and watersheds in {roi} region')
                print('----------')
                wtd_lp_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                                    zone_fields=wtd_cur_fields[0],
                                                                    in_class_features=lakes_fc,
                                                                    out_table=wtd_merge_lp_table_path,
                                                                    class_fields='Ftype',
                                                                    out_units="SQUARE_METERS"
                                                                    )
                # Add region and cat id fields
                arcpy.AlterField_management(wtd_lp_tabint,'PERCENTAGE','wtd_lake_per','wtd_lake_per')
                arcpy.AlterField_management(wtd_lp_tabint,'AREA','wtd_lake_area_sqm','wtd_lake_area_sqm')
                arcpy.AddField_management(wtd_lp_tabint, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[1], field_type='TEXT')
                arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[2], field_type='TEXT')
                wtdlpfields = [f.name for f in arcpy.ListFields(wtd_lp_tabint)]
                print (wtdlpfields)
                with arcpy.da.UpdateCursor(wtd_lp_tabint, wtdlpfields) as cur:
                    for row in cur:
                        strval = str(row[1])
                        row[5] = roi
                        row[6] = strval.replace('.0','')
                        row[7] = roi +'_'+ strval.replace(".0","")
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)

                # Append watershed lakes ponds table to list
                wtd_lp_tabint_tables.append(wtd_lp_tabint)

                # Percent glaciers using Tabulate Intersection for watersheds
                print(f'Begin tabulate intersection between {glac_fc} and watersheds in {roi} region')
                print('----------')
                wtd_glac_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                                    zone_fields=wtd_cur_fields[0],
                                                                    in_class_features=glac_fc,
                                                                    out_table=wtd_merge_glac_table_path,
                                                                    class_fields='O1Region',
                                                                    out_units="SQUARE_METERS"
                                                                    )
                # Add region and cat id fields
                arcpy.AlterField_management(wtd_glac_tabint,'PERCENTAGE','wtd_glacier_per','wtd_glacier_per')
                arcpy.AlterField_management(wtd_glac_tabint,'AREA','wtd_glacier_area_sqm','wtd_glacier_area_sqm')
                arcpy.AddField_management(wtd_glac_tabint, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[1], field_type='TEXT')
                arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[2], field_type='TEXT')
                wtdglacfields = [f.name for f in arcpy.ListFields(wtd_glac_tabint)]
                print (wtdglacfields)
                with arcpy.da.UpdateCursor(wtd_glac_tabint, wtdglacfields) as cur:
                    for row in cur:
                        strval = str(row[1])
                        row[5] = roi
                        row[6] = strval.replace('.0','')
                        row[7] = roi +'_'+ strval.replace(".0","")
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed percent glacier table to list
                wtd_glac_tabint_tables.append(wtd_glac_tabint)

                # Tabulate Area with wetlands grid and watersheds
                print(f'Begin tabulate intersection between {wet_rast} and watersheds in {roi} region')
                print('----------')
                # Wetlands tabulate area for watersheds
                wtd_per_wet_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                              zone_field= wtd_cur_fields[0],
                                                              in_class_data=wet_rast,
                                                              class_field="Value",
                                                              out_table=wtd_merge_wetlands_table_path
                                                              )
                # Add region and percent wet fields
                arcpy.AlterField_management(wtd_per_wet_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
                arcpy.AddField_management(wtd_per_wet_tabarea, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_per_wet_tabarea, 'wtd_wet_per', field_type='Float')
                arcpy.AddField_management(wtd_per_wet_tabarea, wtd_cur_fields[0], field_type='TEXT')
                arcpy.AddField_management(wtd_per_wet_tabarea, wtd_cur_fields[2], field_type='TEXT')
                wtdwetfields = [f.name for f in arcpy.ListFields(wtd_per_wet_tabarea)]
                print (wtdwetfields)
                with arcpy.da.UpdateCursor(wtd_per_wet_tabarea, wtdwetfields) as cur:
                    for row in cur:
                        strval = str(row[1])
                        row[4] = roi
                        row[5] = row[3]/(row[3]+row[2])*100
                        row[6] = strval.replace('.0','')
                        row[7] = roi +'_'+ strval.replace(".0","")
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Drop UPPERCASE field form tab area
                arcpy.DeleteField_management(wtd_per_wet_tabarea,'CAT_ID_TXT_DEL')
                # Append watershed percent wetlands table to list
                wtd_wet_taba_tables.append(wtd_per_wet_tabarea)

                # # Percent North Tabulate Area for catchments
                # cat_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= cats, zone_field='cat_ID_con',
                #                                             in_class_data=nor_rast,"Value",
                #                                             out_table=cat_pernorth_table_path)

                # # Add and calculate region identifier field for catchment table
                # arcpy.AlterField_management(cat_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
                # arcpy.AddField_management(cat_per_north_tabarea, 'region', field_type='TEXT')
                # arcpy.AddField_management(cat_per_north_tabarea, 'cat_north_per', field_type='Float')
                # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[0], field_type='TEXT')
                # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[2], field_type='TEXT')
                # catnorfields = [f.name for f in arcpy.ListFields(cat_per_north_tabarea)]
                # print (catnorfields)
                # with arcpy.da.UpdateCursor(cat_per_north_tabarea,catnorfields) as cur:
                #     for row in cur:
                #         strval = str(row[1])
                #         row[4] = roi
                #         row[5] = row[3]/(row[3]+row[2])*100
                #         row[6] = strval.replace('.0','')
                #         row[7] = roi +'_'+ strval.replace(".0","")
                #         # Update
                #         cur.updateRow(row)
                #     del(row)
                # del(cur)
                # Drop UPPERCASE field form tab area
                # arcpy.DeleteField_management(cat_per_north_tabarea,'CAT_ID_TXT_DEL')
                # # Append catchment percent north table to list
                # cat_pernorth_taba_tables.append(cat_per_north_tabarea)

                tabarea_stop = time.time()
                tabarea_time = int (tabarea_stop - tabarea_start)
                print(f'Tabulate area/intersections for {roi} Elapsed time: ({datetime.timedelta(seconds=tabarea_time)})')
                print('----------')

            except:
                e = sys.exc_info()[1]
                print(e.args[0])
                arcpy.AddError(e.args[0])

            iter_stop = time.time()
            iter_time = int(iter_stop - iteration_start)
            print(f'All Covariates for {roi} completed. Elapsed time: ({datetime.timedelta(seconds=iter_time)})')
            print('----------')

    else:
        print(f'Region {str(roi)} not found in {region}')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')



Cook_Inlet using data from D:\\GIS\\AKSSF\Cook_Inlet folder
Cook_Inlet in ['Cook_Inlet', 'Copper_River'] AKSSF list, using cat_fields ['cat_ID_txt', 'NHDPlusID', 'cat_ID_con'] and watershed fields ['cat_ID_txt', 'cat_ID', 'cat_ID_con']
----------
Adding cat_ID_txt field to catchment dataset D:\\GIS\\AKSSF_land_met\AKSSF_land_met.gdb\Cook_Inlet_cats_intersect
----------
Adding cat_ID_con field to catchment dataset D:\\GIS\\AKSSF_land_met\AKSSF_land_met.gdb\Cook_Inlet_cats_intersect
----------
Merged watershed dataset wtds_merge found
----------
Adding cat_ID_txt field to watershed dataset D:\\GIS\\AKSSF_land_met\AKSSF_land_met.gdb\Cook_Inlet_wtds_merge
----------
Adding cat_ID_con field to watershed dataset D:\\GIS\\AKSSF_land_met\AKSSF_land_met.gdb\Cook_Inlet_wtds_merge
----------
Calculating topographic metrics for catchments & watersheds of interest in Cook_Inlet region
----------
Geodatabase: ['D:\\\\GIS\\\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb']
----------
Watershed Merge: D:\\GIS\\AKS

## Drop unnecessary fields and rename as needed from merged tables.
- Create Key value dictionary and use update cursor to rename fields.

In [7]:
# Table names/paths
wtd_per_north_table_out = os.path.join(outgdb, 'AKSSF_wtd_north_per')
cat_elev_table_out = os.path.join(outgdb,'AKSSF_cat_elev')
cat_slope_table_out = os.path.join(outgdb,'AKSSF_cat_slope')
wtd_elev_table_out = os.path.join(outgdb, 'AKSSF_wtd_elev')
wtd_per_glac_table_out = os.path.join(outgdb, 'AKSSF_wtd_glacier_per')
wtd_per_lp_table_out = os.path.join(outgdb, 'AKSSF_wtd_lakepond_per')
wtd_slope_table_out = os.path.join(outgdb, 'AKSSF_wtd_slope')
wtd_wet_table_out = os.path.join(outgdb, 'AKSSF_wtd_wetland_per')

# Merge all regional tables together
outtables = []
wtd_per_north = arcpy.Merge_management(wtd_pernorth_taba_tables, wtd_per_north_table_out)
arcpy.AlterField_management(wtd_per_north,"VALUE_0","non_north_area","non_north_area")
arcpy.AlterField_management(wtd_per_north,"VALUE_1","north_area","north_area")
outtables.append(wtd_per_north)
cat_elev = arcpy.Merge_management(cat_elev_ztables, cat_elev_table_out)
outtables.append(cat_elev)
wtd_elev = arcpy.Merge_management(wtd_elev_ztables, wtd_elev_table_out)
outtables.append(wtd_elev)
wtd_slope = arcpy.Merge_management(wtd_slope_ztables, wtd_slope_table_out)
outtables.append(wtd_slope)
cat_slope = arcpy.Merge_management(cat_slope_ztables, cat_slope_table_out)
outtables.append(cat_slope)
wtd_wet = arcpy.Merge_management(wtd_wet_taba_tables, wtd_wet_table_out)
arcpy.AlterField_management(wtd_wet,"VALUE_0","non_wetland_area","non_wetland_area")
arcpy.AlterField_management(wtd_wet,"VALUE_1","wetland_area","wetland_area")
outtables.append(wtd_wet)
wtd_glac = arcpy.Merge_management(wtd_glac_tabint_tables, wtd_per_glac_table_out)
outtables.append(wtd_glac)
wtd_lp = arcpy.Merge_management(wtd_lp_tabint_tables, wtd_per_lp_table_out)
outtables.append(wtd_lp)
print ('Tables merged')
print('----------')


Tables merged
----------


In [8]:
#Set up field dictionary
elevDict = { 'ZONE_CODE': ('cat_elev_ZONE_CODE', 'wtd_elev_ZONE_CODE'),
         'COUNT': ('cat_elev_COUNT', 'wtd_elev_COUNT'),
          'AREA': ('cat_elev_AREA', 'wtd_elev_AREA'),
          'MIN': ('cat_elev_MIN', 'wtd_elev_MIN'),
          'MAX': ('cat_elev_MAX', 'wtd_elev_MAX'),
          'RANGE': ('cat_elev_RANGE', 'wtd_elev_RANGE'),
          'MEAN': ('cat_elev_MEAN', 'wtd_elev_MEAN'),
          'STD': ('cat_elev_STD', 'wtd_elev_STD'),
          'SUM': ('cat_elev_SUM', 'wtd_elev_SUM'),
          'VARIETY': ('cat_elev_VARIETY', 'wtd_elev_VARIETY'),
          'MAJORITY': ('cat_elev_MAJORITY', 'wtd_elev_MAJORITY'),
          'MINORITY': ('cat_elev_MINORITY', 'wtd_elev_MINORITY'),
          'MEDIAN': ('cat_elev_MEDIAN', 'wtd_elev_MEDIAN'),
          'PCT90': ('cat_elev_PCT90', 'wtd_elev_PCT90')
         }

slopeDict = { 'ZONE_CODE': ('cat_slope_ZONE_CODE', 'wtd_slope_ZONE_CODE'),
         'COUNT': ('cat_slope_COUNT', 'wtd_slope_COUNT'),
          'AREA': ('cat_slope_AREA', 'wtd_slope_AREA'),
          'MIN': ('cat_slope_MIN', 'wtd_slope_MIN'),
          'MAX': ('cat_slope_MAX', 'wtd_slope_MAX'),
          'RANGE': ('cat_slope_RANGE', 'wtd_slope_RANGE'),
          'MEAN': ('cat_slope_MEAN', 'wtd_slope_MEAN'),
          'STD': ('cat_slope_STD', 'wtd_slope_STD'),
          'SUM': ('cat_slope_SUM', 'wtd_slope_SUM'),
          'VARIETY': ('cat_slope_VARIETY', 'wtd_slope_VARIETY'),
          'MAJORITY': ('cat_slope_MAJORITY', 'wtd_slope_MAJORITY'),
          'MINORITY': ('cat_slope_MINORITY', 'wtd_slope_MINORITY'),
          'MEDIAN': ('cat_slope_MEDIAN', 'wtd_slope_MEDIAN'),
          'PCT90': ('cat_slope_PCT90', 'wtd_slope_PCT90')
         }

# Rename fields for elevation tables
for field in arcpy.ListFields(wtd_elev):
    keyval = field.name
    if keyval in elevDict:
        newname = elevDict[keyval][1]
        newalias = elevDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_elev, keyval, newname, newalias)

for field in arcpy.ListFields(cat_elev):
    keyval = field.name
    if keyval in elevDict:
        newname = elevDict[keyval][0]
        newalias = elevDict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_elev, keyval, newname, newalias)

# Rename fields for slope tables
for field in arcpy.ListFields(wtd_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][1]
        newalias = slopeDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_slope, keyval, newname, newalias)

for field in arcpy.ListFields(cat_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][0]
        newalias = slopeDict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_slope, keyval, newname, newalias)

ZONE_CODE wtd_elev_ZONE_CODE
COUNT wtd_elev_COUNT
AREA wtd_elev_AREA
MIN wtd_elev_MIN
MAX wtd_elev_MAX
RANGE wtd_elev_RANGE
MEAN wtd_elev_MEAN
STD wtd_elev_STD
SUM wtd_elev_SUM
VARIETY wtd_elev_VARIETY
MAJORITY wtd_elev_MAJORITY
MINORITY wtd_elev_MINORITY
MEDIAN wtd_elev_MEDIAN
PCT90 wtd_elev_PCT90
ZONE_CODE cat_elev_ZONE_CODE
COUNT cat_elev_COUNT
AREA cat_elev_AREA
MIN cat_elev_MIN
MAX cat_elev_MAX
RANGE cat_elev_RANGE
MEAN cat_elev_MEAN
STD cat_elev_STD
SUM cat_elev_SUM
VARIETY cat_elev_VARIETY
MAJORITY cat_elev_MAJORITY
MINORITY cat_elev_MINORITY
MEDIAN cat_elev_MEDIAN
PCT90 cat_elev_PCT90
ZONE_CODE wtd_slope_ZONE_CODE
COUNT wtd_slope_COUNT
AREA wtd_slope_AREA
MIN wtd_slope_MIN
MAX wtd_slope_MAX
RANGE wtd_slope_RANGE
MEAN wtd_slope_MEAN
STD wtd_slope_STD
SUM wtd_slope_SUM
MEDIAN wtd_slope_MEDIAN
PCT90 wtd_slope_PCT90
ZONE_CODE cat_slope_ZONE_CODE
COUNT cat_slope_COUNT
AREA cat_slope_AREA
MIN cat_slope_MIN
MAX cat_slope_MAX
RANGE cat_slope_RANGE
MEAN cat_slope_MEAN
STD cat_slope_STD


In [10]:
# # Join watershed tables using cat_ID_con and drop unecessary fields
# wtdtables = ['D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb\AKSSF_wtd_north_per',
#              'D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb\AKSSF_wtd_elev',
#              'D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb\AKSSF_wtd_slope',
#              'D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb\AKSSF_wtd_wetland_per',
#              'D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb\AKSSF_wtd_glacier_per',
#              'D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb\AKSSF_wtd_lakepond_per']
#
# dropfields = ['OBJECTID','cat_ID_txt','region','cat_ID','cat_elev_ZONE_CODE','wtd_elev_ZONE_CODE','cat_ID_con']
# joinfields = []
# wtd_cov_table = arcpy.Copy_management(wtdtables[0],os.path.join(outgdb,'AKSSF_wtd_covariates'))
# for table in wtdtables[1:]:
#     joinfields = []
#     fields = arcpy.ListFields(table)
#     for field in fields:
#         if field.name not in dropfields:
#             print(field.name)
#             joinfields.append(field.name)
#         else:
#             print (f'{table} - dropping {field.name}')
#     print(joinfields)
#     arcpy.JoinField_management(wtd_cov_table,'cat_ID_con',table,'cat_ID_con',joinfields)
# outtables.append(wtd_cov_table)

In [9]:
for table in outtables:
    print (arcpy.Describe(table).basename)

AKSSF_wtd_north_per
AKSSF_cat_elev
AKSSF_wtd_elev
AKSSF_wtd_slope
AKSSF_cat_slope
AKSSF_wtd_wetland_per
AKSSF_wtd_glacier_per
AKSSF_wtd_lakepond_per


In [10]:
# Export copies of dbf tables as csv
outdir = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers"
for table in outtables:
    tablename = arcpy.Describe(table).basename + ".csv"
    tablepath = os.path.join(outdir,tablename)
    print( tablepath)
    arcpy.conversion.TableToTable(table, outdir, tablename)

C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_wtd_north_per.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_cat_elev.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_wtd_elev.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_wtd_slope.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_cat_slope.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_wtd_wetland_per.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_wtd_glacier_per.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_wtd_lakepond_per.csv


In [11]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format # only display 2 decimal places
# list to store covariate data frames
dfs = []

In [12]:
# Make catchment elev df
cat_df = pd.DataFrame()
cat_field_list = []
for field in arcpy.ListFields(cat_elev):
    cat_field_list.append(field.name)
cat_elev_arr = arcpy.da.TableToNumPyArray(cat_elev,cat_field_list)
cat_df = pd.DataFrame(cat_elev_arr)
cat_df = cat_df.drop(["OBJECTID","cat_elev_ZONE_CODE"],axis=1)
cat_df = cat_df.set_index('cat_ID_con')
dfs.append(cat_df)
cat_df


,cat_ID_txt,cat_elev_COUNT,cat_elev_AREA,cat_elev_MIN,cat_elev_MAX,cat_elev_RANGE,cat_elev_MEAN,cat_elev_STD,cat_elev_SUM,cat_elev_VARIETY,cat_elev_MAJORITY,cat_elev_MINORITY,cat_elev_MEDIAN,cat_elev_PCT90,region
cat_ID_con,,,,,,,,,,,,,,,
Cook_Inlet_75004300006312,75004300006312,14385.00,1438500.00,59,98,39,76.26,9.35,1097028.00,40,69,98,74,90,Cook_Inlet
Cook_Inlet_75004300001906,75004300001906,3033.00,303300.00,3,142,139,27.82,21.10,84384.00,126,9,78,21,52,Cook_Inlet
Cook_Inlet_75004300000100,75004300000100,19699.00,1969900.00,33,78,45,58.66,8.20,1155599.00,46,65,33,59,68,Cook_Inlet
Cook_Inlet_75004300004983,75004300004983,28386.00,2838600.00,21,478,457,264.67,100.37,7512826.00,456,336,29,282,387,Cook_Inlet
Cook_Inlet_75004300004332,75004300004332,11742.00,1174200.00,3,70,67,37.62,17.32,441788.00,68,35,70,38,61,Cook_Inlet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Prince_William_Sound_43933,43933,3776.00,377600.00,0,512,512,222.92,164.12,841739.00,512,4,32,222,455,Prince_William_Sound
Prince_William_Sound_43973,43973,2363.00,236300.00,1,362,361,52.16,67.87,123252.00,258,4,141,20,131,Prince_William_Sound
Prince_William_Sound_44553,44553,3888.00,388800.00,0,35,35,4.44,3.49,17249.00,34,5,17,4,8,Prince_William_Sound


In [13]:
# Make catchment slope df
cat_sl_df = pd.DataFrame()
cat_sl_field_list = []
for field in arcpy.ListFields(cat_slope):
    cat_sl_field_list.append(field.name)
cat_sl_arr = arcpy.da.TableToNumPyArray(cat_slope, cat_sl_field_list)
cat_sl_df = pd.DataFrame(cat_sl_arr)
cat_sl_df = cat_sl_df.drop(["OBJECTID", "cat_slope_ZONE_CODE"],axis=1)
cat_sl_df = cat_sl_df.set_index('cat_ID_con')
dfs.append(cat_sl_df)
cat_sl_df

,cat_ID_txt,cat_slope_COUNT,cat_slope_AREA,cat_slope_MIN,cat_slope_MAX,cat_slope_RANGE,cat_slope_MEAN,cat_slope_STD,cat_slope_SUM,cat_slope_MEDIAN,cat_slope_PCT90,region
cat_ID_con,,,,,,,,,,,,
Cook_Inlet_75004300006312,75004300006312,14385.00,1438500.00,0.00,19.77,19.77,3.53,2.74,50717.83,3.03,6.75,Cook_Inlet
Cook_Inlet_75004300001906,75004300001906,3033.00,303300.00,0.00,36.33,36.33,9.05,7.85,27461.82,6.00,21.17,Cook_Inlet
Cook_Inlet_75004300000100,75004300000100,19699.00,1969900.00,0.00,40.58,40.58,3.93,3.83,77345.54,3.03,8.71,Cook_Inlet
Cook_Inlet_75004300004983,75004300004983,28386.00,2838600.00,0.00,56.34,56.34,17.85,10.56,506737.12,16.35,32.45,Cook_Inlet
Cook_Inlet_75004300004332,75004300004332,11742.00,1174200.00,0.00,38.19,38.19,5.76,5.65,67677.01,3.44,13.66,Cook_Inlet
...,...,...,...,...,...,...,...,...,...,...,...,...
Prince_William_Sound_43933,43933,3776.00,377600.00,0.00,61.45,61.45,29.48,14.86,111305.62,34.15,44.44,Prince_William_Sound
Prince_William_Sound_43973,43973,2363.00,236300.00,0.00,57.29,57.29,14.32,12.91,33834.77,11.35,30.90,Prince_William_Sound
Prince_William_Sound_44553,44553,3888.00,388800.00,0.00,22.45,22.45,4.22,3.66,16407.55,3.37,8.56,Prince_William_Sound


In [14]:
# Make watershed elev df
wtd_df = pd.DataFrame()
wtd_field_list = []
for field in arcpy.ListFields(wtd_elev):
    wtd_field_list.append(field.name)
wtd_elev_arr = arcpy.da.TableToNumPyArray(wtd_elev,wtd_field_list)
wtd_df = pd.DataFrame(wtd_elev_arr)
wtd_df = wtd_df.drop(["OBJECTID","wtd_elev_ZONE_CODE"],axis=1)
wtd_df = wtd_df.set_index('cat_ID_con')
dfs.append(wtd_df)
wtd_df

,cat_ID_txt,wtd_elev_COUNT,wtd_elev_AREA,wtd_elev_MIN,wtd_elev_MAX,wtd_elev_RANGE,wtd_elev_MEAN,wtd_elev_STD,wtd_elev_SUM,wtd_elev_VARIETY,wtd_elev_MAJORITY,wtd_elev_MINORITY,wtd_elev_MEDIAN,wtd_elev_PCT90,region
cat_ID_con,,,,,,,,,,,,,,,
Cook_Inlet_75004300006312,75004300006312,1543049.00,154304900.00,59,569,510,250.91,117.73,387159045.00,511,126,569,227,431,Cook_Inlet
Cook_Inlet_75004300001906,75004300001906,918706.00,91870600.00,3,771,768,401.25,119.88,368626449.00,769,377,771,402,546,Cook_Inlet
Cook_Inlet_75004300000100,75004300000100,289619.00,28961900.00,33,417,384,156.21,85.74,45241203.00,385,91,417,128,294,Cook_Inlet
Cook_Inlet_75004300004983,75004300004983,152611.00,15261100.00,21,1472,1451,671.93,334.26,102543640.00,1449,336,29,648,1169,Cook_Inlet
Cook_Inlet_75004300004332,75004300004332,3301259.00,330125900.00,3,629,626,202.57,144.51,668745483.00,627,110,629,143,454,Cook_Inlet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Prince_William_Sound_43933,43933,323575.00,32357500.00,0,1474,1474,622.17,301.53,201317499.00,1448,200,1355,646,1009,Prince_William_Sound
Prince_William_Sound_43973,43973,166989.00,16698900.00,1,1202,1201,409.04,242.26,68305473.00,1194,254,1164,383,755,Prince_William_Sound
Prince_William_Sound_44553,44553,502923.00,50292300.00,0,1697,1697,592.86,356.45,298163889.00,1688,45,1493,587,1075,Prince_William_Sound


In [15]:
# Make watershed slope df
wtd_sl_df = pd.DataFrame()
wtd_sl_field_list = []
for field in arcpy.ListFields(wtd_slope):
    wtd_sl_field_list.append(field.name)
wtd_sl_arr = arcpy.da.TableToNumPyArray(wtd_slope, wtd_sl_field_list)
wtd_sl_df = pd.DataFrame(wtd_sl_arr)
wtd_sl_df = wtd_sl_df.drop(["OBJECTID", "wtd_slope_ZONE_CODE"],axis=1)
wtd_sl_df = wtd_sl_df.set_index('cat_ID_con')
dfs.append(wtd_sl_df)
wtd_sl_df

,cat_ID_txt,wtd_slope_COUNT,wtd_slope_AREA,wtd_slope_MIN,wtd_slope_MAX,wtd_slope_RANGE,wtd_slope_MEAN,wtd_slope_STD,wtd_slope_SUM,wtd_slope_MEDIAN,wtd_slope_PCT90,region
cat_ID_con,,,,,,,,,,,,
Cook_Inlet_75004300006312,75004300006312,1543049.00,154304900.00,0.00,41.29,41.29,5.31,4.22,8191560.61,4.10,10.90,Cook_Inlet
Cook_Inlet_75004300001906,75004300001906,918706.00,91870600.00,0.00,52.67,52.67,8.46,10.17,7775816.06,4.10,26.70,Cook_Inlet
Cook_Inlet_75004300000100,75004300000100,289619.00,28961900.00,0.00,40.58,40.58,3.96,3.63,1146766.58,3.37,8.50,Cook_Inlet
Cook_Inlet_75004300004983,75004300004983,152611.00,15261100.00,0.00,66.25,66.25,21.01,12.17,3206958.49,19.21,38.54,Cook_Inlet
Cook_Inlet_75004300004332,75004300004332,3301259.00,330125900.00,0.00,43.23,43.23,4.65,4.30,15351663.65,3.43,9.96,Cook_Inlet
...,...,...,...,...,...,...,...,...,...,...,...,...
Prince_William_Sound_43933,43933,323575.00,32357500.00,0.00,77.52,77.52,31.20,12.90,10095389.38,31.48,47.58,Prince_William_Sound
Prince_William_Sound_43973,43973,166989.00,16698900.00,0.00,69.60,69.60,28.83,12.80,4813591.81,30.27,44.42,Prince_William_Sound
Prince_William_Sound_44553,44553,502923.00,50292300.00,0.00,74.11,74.11,24.59,13.51,12368255.48,24.67,42.61,Prince_William_Sound


In [16]:
# Make watershed north df
wtd_n_df = pd.DataFrame()
wtd_n_field_list = []
for field in arcpy.ListFields(wtd_per_north):
    wtd_n_field_list.append(field.name)
wtd_n_arr = arcpy.da.TableToNumPyArray(wtd_per_north,wtd_n_field_list)
wtd_n_df = pd.DataFrame(wtd_n_arr)
wtd_n_df = wtd_n_df.drop("OBJECTID",axis=1)
wtd_n_df = wtd_n_df.set_index('cat_ID_con')
dfs.append(wtd_n_df)
wtd_n_df

,non_north_area,north_area,region,wtd_north_per,cat_ID_txt
cat_ID_con,,,,,
Cook_Inlet_75004300006312,116022600.00,38282300.00,Cook_Inlet,24.81,75004300006312
Cook_Inlet_75004300001906,78243800.00,13626800.00,Cook_Inlet,14.83,75004300001906
Cook_Inlet_75004300000100,21835000.00,7126900.00,Cook_Inlet,24.61,75004300000100
Cook_Inlet_75004300004983,11244800.00,4016300.00,Cook_Inlet,26.32,75004300004983
Cook_Inlet_75004300004332,229068700.00,101057200.00,Cook_Inlet,30.61,75004300004332
...,...,...,...,...,...
Prince_William_Sound_43933,27722700.00,4634800.00,Prince_William_Sound,14.32,43933
Prince_William_Sound_43973,12997000.00,3701900.00,Prince_William_Sound,22.17,43973
Prince_William_Sound_44553,42961800.00,7330500.00,Prince_William_Sound,14.58,44553


In [17]:
# Make watershed wetland df
wtd_wet_df = pd.DataFrame()
wtd_wet_field_list = []
for field in arcpy.ListFields(wtd_wet):
    wtd_wet_field_list.append(field.name)
wtd_wet_arr = arcpy.da.TableToNumPyArray(wtd_wet,wtd_wet_field_list)
wtd_wet_df = pd.DataFrame(wtd_wet_arr)
wtd_wet_df = wtd_wet_df.drop("OBJECTID",axis=1)
wtd_wet_df = wtd_wet_df.set_index('cat_ID_con')
dfs.append(wtd_wet_df)
wtd_wet_df

,non_wetland_area,wetland_area,region,wtd_wet_per,cat_ID_txt
cat_ID_con,,,,,
Cook_Inlet_75004300006312,130773100.00,23531800.00,Cook_Inlet,15.25,75004300006312
Cook_Inlet_75004300001906,73734700.00,18135900.00,Cook_Inlet,19.74,75004300001906
Cook_Inlet_75004300000100,18485400.00,10476500.00,Cook_Inlet,36.17,75004300000100
Cook_Inlet_75004300004983,15150400.00,110700.00,Cook_Inlet,0.73,75004300004983
Cook_Inlet_75004300004332,259180700.00,70945200.00,Cook_Inlet,21.49,75004300004332
...,...,...,...,...,...
Prince_William_Sound_43933,32296200.00,61300.00,Prince_William_Sound,0.19,43933
Prince_William_Sound_43973,16629700.00,69200.00,Prince_William_Sound,0.41,43973
Prince_William_Sound_44553,49886500.00,405800.00,Prince_William_Sound,0.81,44553


In [18]:
# Make watershed lakes df
wtd_lp_df = pd.DataFrame()
wtd_lp_field_list = []
for field in arcpy.ListFields(wtd_lp):
    wtd_lp_field_list.append(field.name)
wtd_lp_arr = arcpy.da.TableToNumPyArray(wtd_lp, wtd_lp_field_list)
wtd_lp_df = pd.DataFrame(wtd_lp_arr)
wtd_lp_df = wtd_lp_df.drop("OBJECTID",axis=1)
wtd_lp_df = wtd_lp_df.set_index('cat_ID_con')
dfs.append(wtd_lp_df)
wtd_lp_df

,cat_ID_txt,FType,wtd_lake_area_sqm,wtd_lake_per,region,cat_ID
cat_ID_con,,,,,,
Cook_Inlet_75000100000004,75000100000004,390,922040.71,0.54,Cook_Inlet,75000100000004
Cook_Inlet_75000100000059,75000100000059,390,314031.76,0.87,Cook_Inlet,75000100000059
Cook_Inlet_75000100000652,75000100000652,390,32219941.31,9.97,Cook_Inlet,75000100000652
Cook_Inlet_75000100000981,75000100000981,390,753288.15,0.68,Cook_Inlet,75000100000981
Cook_Inlet_75000100001919,75000100001919,390,391083.09,0.54,Cook_Inlet,75000100001919
...,...,...,...,...,...,...
Prince_William_Sound_43933,43933,390,484188.75,1.50,Prince_William_Sound,43933
Prince_William_Sound_43973,43973,390,12804.67,0.08,Prince_William_Sound,43973
Prince_William_Sound_44553,44553,390,1216.19,0.00,Prince_William_Sound,44553


In [19]:
# Make watershed glacier df
wtd_glac_df = pd.DataFrame()
wtd_glac_field_list = []
for field in arcpy.ListFields(wtd_glac):
    wtd_glac_field_list.append(field.name)
wtd_glac_arr = arcpy.da.TableToNumPyArray(wtd_glac, wtd_glac_field_list)
wtd_glac_df = pd.DataFrame(wtd_glac_arr)
wtd_glac_df = wtd_glac_df.drop("OBJECTID",axis=1)
wtd_glac_df = wtd_glac_df.set_index('cat_ID_con')
dfs.append(wtd_glac_df)
wtd_glac_df

,cat_ID_txt,O1Region,wtd_glacier_area_sqm,wtd_glacier_per,region,cat_ID
cat_ID_con,,,,,,
Cook_Inlet_75000100002463,75000100002463,1,170835.62,0.05,Cook_Inlet,75000100002463
Cook_Inlet_75000100003094,75000100003094,1,170835.62,0.07,Cook_Inlet,75000100003094
Cook_Inlet_75000200000021,75000200000021,1,1719257.23,0.22,Cook_Inlet,75000200000021
Cook_Inlet_75000200002162,75000200002162,1,1719257.23,1.59,Cook_Inlet,75000200002162
Cook_Inlet_75000200003732,75000200003732,1,1719257.23,0.40,Cook_Inlet,75000200003732
...,...,...,...,...,...,...
Kodiak_128685,128685,1,404722.01,0.39,Kodiak,128685
Prince_William_Sound_43185,43185,1,16637956.23,69.46,Prince_William_Sound,43185
Prince_William_Sound_43933,43933,1,918368.25,2.84,Prince_William_Sound,43933


## Merge all covariate dataframes together and drop unnecessary columns
 * Recalculate cat_ID as float64 type
 * Reorder columns
 * Export final csv


In [20]:
# Merge all data frames together
import numpy as np
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on='cat_ID_con',how="outer"), dfs)
#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)
df_final.columns = list(uniquify(df_final))
#List of final columns in the order to output
final_cols = ['cat_ID_txt','cat_ID','region', 'cat_slope_COUNT', 'cat_slope_AREA', 'cat_slope_MIN', 'cat_slope_MAX',
              'cat_slope_RANGE','cat_slope_MEAN', 'cat_slope_STD', 'cat_slope_SUM', 'cat_slope_MEDIAN', 'cat_slope_PCT90',
              'cat_elev_COUNT', 'cat_elev_AREA', 'cat_elev_MIN', 'cat_elev_MAX', 'cat_elev_RANGE', 'cat_elev_MEAN', 'cat_elev_STD',
              'cat_elev_SUM', 'cat_elev_VARIETY', 'cat_elev_MAJORITY', 'cat_elev_MINORITY', 'cat_elev_MEDIAN', 'cat_elev_PCT90',
              'wtd_elev_COUNT', 'wtd_elev_AREA', 'wtd_elev_MIN', 'wtd_elev_MAX', 'wtd_elev_RANGE', 'wtd_elev_MEAN',
              'wtd_elev_STD', 'wtd_elev_SUM', 'wtd_elev_VARIETY', 'wtd_elev_MAJORITY', 'wtd_elev_MINORITY',
              'wtd_elev_MEDIAN', 'wtd_elev_PCT90', 'wtd_slope_COUNT', 'wtd_slope_AREA', 'wtd_slope_MIN', 'wtd_slope_MAX',
              'wtd_slope_RANGE', 'wtd_slope_MEAN', 'wtd_slope_STD', 'wtd_slope_SUM', 'wtd_slope_MEDIAN', 'wtd_slope_PCT90',
              'non_north_area', 'north_area', 'wtd_north_per', 'non_wetland_area', 'wetland_area', 'wtd_wet_per',
              'wtd_lake_area_sqm', 'wtd_lake_per', 'wtd_glacier_area_sqm', 'wtd_glacier_per' ]
#Create list of duplicate column names and drop
drop_cols = ['cat_ID_txt_y', 'region_y', 'cat_ID_txt_x_2', 'region_x_2', 'region_y_2', 'cat_ID_txt_y_2', 'region_x_3',
             'cat_ID_txt_x_3', 'cat_ID_txt_y_3', 'FType', 'region_y_3','cat_ID_txt_x_4', 'O1Region', 'region_x_4',
             'cat_ID_y', 'cat_ID_txt_y_4', 'region_y_4']
df_final.drop(columns=drop_cols, axis = 1, inplace=True)
#rename columns
df_final.rename({'cat_ID_txt_x':'cat_ID_txt','cat_ID_x':'cat_ID','region_x':'region'},axis=1, inplace=True)
#Recalculate cat_ID
df_final['cat_ID'] = df_final['cat_ID_txt'].astype(np.float64)
# reorder cols
df_final = df_final.reindex(columns=final_cols)
df_final

,cat_ID_txt,cat_ID,region,cat_slope_COUNT,cat_slope_AREA,cat_slope_MIN,cat_slope_MAX,cat_slope_RANGE,cat_slope_MEAN,cat_slope_STD,...,non_north_area,north_area,wtd_north_per,non_wetland_area,wetland_area,wtd_wet_per,wtd_lake_area_sqm,wtd_lake_per,wtd_glacier_area_sqm,wtd_glacier_per
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Cook_Inlet_75004300006312,75004300006312,75004300006312.00,Cook_Inlet,14385.00,1438500.00,0.00,19.77,19.77,3.53,2.74,...,116022600.00,38282300.00,24.81,130773100.00,23531800.00,15.25,35351.21,0.02,NaN,NaN
Cook_Inlet_75004300001906,75004300001906,75004300001906.00,Cook_Inlet,3033.00,303300.00,0.00,36.33,36.33,9.05,7.85,...,78243800.00,13626800.00,14.83,73734700.00,18135900.00,19.74,3155003.18,3.43,NaN,NaN
Cook_Inlet_75004300000100,75004300000100,75004300000100.00,Cook_Inlet,19699.00,1969900.00,0.00,40.58,40.58,3.93,3.83,...,21835000.00,7126900.00,24.61,18485400.00,10476500.00,36.17,35873.50,0.12,NaN,NaN
Cook_Inlet_75004300004983,75004300004983,75004300004983.00,Cook_Inlet,28386.00,2838600.00,0.00,56.34,56.34,17.85,10.56,...,11244800.00,4016300.00,26.32,15150400.00,110700.00,0.73,145404.68,0.95,445858.96,2.92
Cook_Inlet_75004300004332,75004300004332,75004300004332.00,Cook_Inlet,11742.00,1174200.00,0.00,38.19,38.19,5.76,5.65,...,229068700.00,101057200.00,30.61,259180700.00,70945200.00,21.49,462571.33,0.14,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Prince_William_Sound_43933,43933,43933.00,Prince_William_Sound,3776.00,377600.00,0.00,61.45,61.45,29.48,14.86,...,27722700.00,4634800.00,14.32,32296200.00,61300.00,0.19,484188.75,1.50,918368.25,2.84
Prince_William_Sound_43973,43973,43973.00,Prince_William_Sound,2363.00,236300.00,0.00,57.29,57.29,14.32,12.91,...,12997000.00,3701900.00,22.17,16629700.00,69200.00,0.41,12804.67,0.08,NaN,NaN
Prince_William_Sound_44553,44553,44553.00,Prince_William_Sound,3888.00,388800.00,0.00,22.45,22.45,4.22,3.66,...,42961800.00,7330500.00,14.58,49886500.00,405800.00,0.81,1216.19,0.00,23522394.82,46.77


In [21]:
# Export merged dataframe to csv
cov_csv_out = os.path.join(outdir,'AKSSF_Covariates.csv')
df_final.to_csv(cov_csv_out, encoding = 'utf-8')
print('Export all covariates dataframe to csv complete')


Export all covariates dataframe to csv complete


In [22]:
bbay_df = df_final.filter(like='Bristol_Bay', axis = 0)
bbay_df

,cat_ID_txt,cat_ID,region,cat_slope_COUNT,cat_slope_AREA,cat_slope_MIN,cat_slope_MAX,cat_slope_RANGE,cat_slope_MEAN,cat_slope_STD,...,non_north_area,north_area,wtd_north_per,non_wetland_area,wetland_area,wtd_wet_per,wtd_lake_area_sqm,wtd_lake_per,wtd_glacier_area_sqm,wtd_glacier_per
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Bristol_Bay_1023044,1023044,1023044.00,Bristol_Bay,5412.00,541200.00,0.00,18.68,18.68,3.15,3.16,...,2810795900.00,454473400.00,13.92,3208126900.00,57142400.00,1.75,1219325461.61,37.34,8152617.83,0.25
Bristol_Bay_2041471,2041471,2041471.00,Bristol_Bay,2039.00,203900.00,0.00,16.33,16.33,2.66,3.07,...,922316600.00,266516800.00,22.42,1188195000.00,639000.00,0.05,786404.78,0.07,328048246.94,27.59
Bristol_Bay_2065755,2065755,2065755.00,Bristol_Bay,2731.00,273100.00,0.00,40.04,40.04,9.66,8.71,...,237064600.00,68862800.00,22.51,305695600.00,231800.00,0.08,16708795.84,5.46,NaN,NaN
Bristol_Bay_2065914,2065914,2065914.00,Bristol_Bay,4356.00,435600.00,0.00,18.58,18.58,4.63,3.35,...,10904800.00,2065000.00,15.92,12689000.00,280800.00,2.17,817016.06,6.30,NaN,NaN
Bristol_Bay_2066924,2066924,2066924.00,Bristol_Bay,6246.00,624600.00,0.00,44.27,44.27,17.71,12.82,...,24587800.00,5114700.00,17.22,29694400.00,8100.00,0.03,998848.47,3.36,1498522.87,5.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Bristol_Bay_5030704,5030704,5030704.00,Bristol_Bay,4048.00,404800.00,0.00,27.04,27.04,6.25,5.76,...,32215300.00,12241300.00,27.54,44445300.00,11300.00,0.03,1015804.10,2.28,NaN,NaN
Bristol_Bay_5032474,5032474,5032474.00,Bristol_Bay,1233.00,123300.00,0.00,19.34,19.34,3.24,3.53,...,60968200.00,13865600.00,18.53,74746500.00,87300.00,0.12,7106548.38,9.50,NaN,NaN
Bristol_Bay_5033094,5033094,5033094.00,Bristol_Bay,3462.00,346200.00,0.00,19.06,19.06,3.54,3.52,...,196594700.00,51343000.00,20.71,246494400.00,1443300.00,0.58,9847561.83,3.97,NaN,NaN


In [23]:
kod_df = df_final.filter(like='Kodiak', axis = 0)
kod_df

,cat_ID_txt,cat_ID,region,cat_slope_COUNT,cat_slope_AREA,cat_slope_MIN,cat_slope_MAX,cat_slope_RANGE,cat_slope_MEAN,cat_slope_STD,...,non_north_area,north_area,wtd_north_per,non_wetland_area,wetland_area,wtd_wet_per,wtd_lake_area_sqm,wtd_lake_per,wtd_glacier_area_sqm,wtd_glacier_per
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Kodiak_48267,48267,48267.00,Kodiak,7607.00,760700.00,0.00,33.50,33.50,5.45,6.34,...,61868800.00,18689300.00,23.20,76844500.00,3713600.00,4.61,13903654.69,17.26,NaN,NaN
Kodiak_49617,49617,49617.00,Kodiak,1470.00,147000.00,0.00,25.28,25.28,4.70,4.84,...,226344300.00,67788600.00,23.05,270508600.00,23624300.00,8.03,18815463.74,6.40,NaN,NaN
Kodiak_50197,50197,50197.00,Kodiak,819.00,81900.00,0.00,18.58,18.58,6.03,4.17,...,37915200.00,11157000.00,22.74,47588400.00,1483800.00,3.02,5285341.13,10.77,NaN,NaN
Kodiak_64593,64593,64593.00,Kodiak,2080.00,208000.00,0.00,49.26,49.26,17.39,10.95,...,49545800.00,16021800.00,24.44,64563400.00,1004200.00,1.53,6437073.99,9.82,NaN,NaN
Kodiak_72144,72144,72144.00,Kodiak,657.00,65700.00,0.68,29.27,28.59,13.81,5.41,...,3014300.00,1194500.00,28.38,4198900.00,9900.00,0.24,5613.24,0.13,NaN,NaN
Kodiak_76954,76954,76954.00,Kodiak,4168.00,416800.00,0.00,40.16,40.16,6.28,6.60,...,20486000.00,10390600.00,33.65,30711700.00,164900.00,0.53,1072121.70,3.47,NaN,NaN
Kodiak_77794,77794,77794.00,Kodiak,4689.00,468900.00,0.00,39.94,39.94,6.34,6.84,...,41022000.00,13776500.00,25.14,54413000.00,385500.00,0.70,1542672.37,2.82,NaN,NaN
Kodiak_90346,90346,90346.00,Kodiak,19618.00,1961800.00,0.00,52.89,52.89,15.72,12.28,...,5316200.00,2212400.00,29.39,7258400.00,270200.00,3.59,247704.21,3.29,NaN,NaN
Kodiak_93176,93176,93176.00,Kodiak,853.00,85300.00,0.00,21.56,21.56,6.48,4.89,...,6908000.00,3679200.00,34.75,10577100.00,10100.00,0.10,2.74,0.00,NaN,NaN


In [24]:
pws_df = df_final.filter(like='Prince', axis = 0)
pws_df

,cat_ID_txt,cat_ID,region,cat_slope_COUNT,cat_slope_AREA,cat_slope_MIN,cat_slope_MAX,cat_slope_RANGE,cat_slope_MEAN,cat_slope_STD,...,non_north_area,north_area,wtd_north_per,non_wetland_area,wetland_area,wtd_wet_per,wtd_lake_area_sqm,wtd_lake_per,wtd_glacier_area_sqm,wtd_glacier_per
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Prince_William_Sound_18457,18457,18457.00,Prince_William_Sound,2358.00,235800.00,0.00,25.44,25.44,7.92,5.82,...,10099600.00,2553800.00,20.18,7474700.00,5178700.00,40.93,787431.48,6.22,NaN,NaN
Prince_William_Sound_26464,26464,26464.00,Prince_William_Sound,10795.00,1079500.00,0.00,48.38,48.38,14.21,9.08,...,2530600.00,1979100.00,43.89,4349700.00,160000.00,3.55,780.68,0.02,NaN,NaN
Prince_William_Sound_28086,28086,28086.00,Prince_William_Sound,2900.00,290000.00,0.00,58.78,58.78,31.82,11.25,...,3734500.00,494000.00,11.68,4206000.00,22500.00,0.53,NaN,NaN,NaN,NaN
Prince_William_Sound_29854,29854,29854.00,Prince_William_Sound,1862.00,186200.00,0.00,30.98,30.98,7.18,7.16,...,10698500.00,7543400.00,41.35,17451000.00,790900.00,4.34,40876.52,0.22,NaN,NaN
Prince_William_Sound_30884,30884,30884.00,Prince_William_Sound,1971.00,197100.00,0.00,31.56,31.56,6.98,6.24,...,5746100.00,2676200.00,31.78,4941100.00,3481200.00,41.33,110508.29,1.31,NaN,NaN
Prince_William_Sound_31865,31865,31865.00,Prince_William_Sound,2456.00,245600.00,0.00,23.15,23.15,7.99,3.83,...,624700.00,265000.00,29.79,425300.00,464400.00,52.20,NaN,NaN,NaN,NaN
Prince_William_Sound_36645,36645,36645.00,Prince_William_Sound,123.00,12300.00,0.00,27.21,27.21,14.89,8.87,...,1908700.00,1686000.00,46.90,3588100.00,6600.00,0.18,NaN,NaN,NaN,NaN
Prince_William_Sound_37815,37815,37815.00,Prince_William_Sound,11468.00,1146800.00,0.00,61.61,61.61,19.71,17.02,...,3919900.00,1347200.00,25.58,5232300.00,34800.00,0.66,575759.86,10.93,NaN,NaN
Prince_William_Sound_38993,38993,38993.00,Prince_William_Sound,4342.00,434200.00,0.00,15.66,15.66,3.49,2.90,...,2962600.00,206000.00,6.50,1687100.00,1481500.00,46.76,83031.97,2.62,NaN,NaN


In [25]:
ci_df = df_final.filter(like='Cook', axis = 0)
ci_df

,cat_ID_txt,cat_ID,region,cat_slope_COUNT,cat_slope_AREA,cat_slope_MIN,cat_slope_MAX,cat_slope_RANGE,cat_slope_MEAN,cat_slope_STD,...,non_north_area,north_area,wtd_north_per,non_wetland_area,wetland_area,wtd_wet_per,wtd_lake_area_sqm,wtd_lake_per,wtd_glacier_area_sqm,wtd_glacier_per
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Cook_Inlet_75004300006312,75004300006312,75004300006312.00,Cook_Inlet,14385.00,1438500.00,0.00,19.77,19.77,3.53,2.74,...,116022600.00,38282300.00,24.81,130773100.00,23531800.00,15.25,35351.21,0.02,NaN,NaN
Cook_Inlet_75004300001906,75004300001906,75004300001906.00,Cook_Inlet,3033.00,303300.00,0.00,36.33,36.33,9.05,7.85,...,78243800.00,13626800.00,14.83,73734700.00,18135900.00,19.74,3155003.18,3.43,NaN,NaN
Cook_Inlet_75004300000100,75004300000100,75004300000100.00,Cook_Inlet,19699.00,1969900.00,0.00,40.58,40.58,3.93,3.83,...,21835000.00,7126900.00,24.61,18485400.00,10476500.00,36.17,35873.50,0.12,NaN,NaN
Cook_Inlet_75004300004983,75004300004983,75004300004983.00,Cook_Inlet,28386.00,2838600.00,0.00,56.34,56.34,17.85,10.56,...,11244800.00,4016300.00,26.32,15150400.00,110700.00,0.73,145404.68,0.95,445858.96,2.92
Cook_Inlet_75004300004332,75004300004332,75004300004332.00,Cook_Inlet,11742.00,1174200.00,0.00,38.19,38.19,5.76,5.65,...,229068700.00,101057200.00,30.61,259180700.00,70945200.00,21.49,462571.33,0.14,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cook_Inlet_75005300022831,75005300022831,75005300022831.00,Cook_Inlet,8750.00,875000.00,0.00,23.06,23.06,4.38,3.59,...,298256000.00,52264700.00,14.91,326175300.00,24345400.00,6.95,2509919.26,0.72,NaN,NaN
Cook_Inlet_75005300011378,75005300011378,75005300011378.00,Cook_Inlet,11021.00,1102100.00,0.00,33.65,33.65,4.14,3.93,...,258832200.00,43364700.00,14.35,264568700.00,37628200.00,12.45,1213504.71,0.40,NaN,NaN
Cook_Inlet_75005400001203,75005400001203,75005400001203.00,Cook_Inlet,657.00,65700.00,0.00,21.41,21.41,4.95,4.66,...,210110000.00,72341000.00,25.61,282451000.00,0.00,0.00,5116356.18,1.81,3868262.78,1.37


In [26]:
cop_df = df_final.filter(like='Copper', axis = 0)
cop_df

,cat_ID_txt,cat_ID,region,cat_slope_COUNT,cat_slope_AREA,cat_slope_MIN,cat_slope_MAX,cat_slope_RANGE,cat_slope_MEAN,cat_slope_STD,...,non_north_area,north_area,wtd_north_per,non_wetland_area,wetland_area,wtd_wet_per,wtd_lake_area_sqm,wtd_lake_per,wtd_glacier_area_sqm,wtd_glacier_per
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Copper_River_75019800000406,75019800000406,75019800000406.00,Copper_River,73105.00,7310500.00,0.00,14.33,14.33,0.89,1.48,...,6451700.00,858800.00,11.75,6396500.00,914000.00,12.50,NaN,NaN,NaN,NaN
Copper_River_75019800010313,75019800010313,75019800010313.00,Copper_River,2912.00,291200.00,0.00,3.44,3.44,0.54,1.11,...,444687800.00,116996000.00,20.83,487125400.00,74558400.00,13.27,20596534.73,3.66,2266169.90,0.40
Copper_River_75019800014348,75019800014348,75019800014348.00,Copper_River,4298.00,429800.00,0.00,16.48,16.48,3.08,2.54,...,31113100.00,4000400.00,11.39,32228500.00,2885000.00,8.22,111418.22,0.32,NaN,NaN
Copper_River_75019800001957,75019800001957,75019800001957.00,Copper_River,932.00,93200.00,0.00,11.67,11.67,3.86,3.09,...,12197400.00,517300.00,4.07,12622500.00,92200.00,0.73,6051.47,0.05,NaN,NaN
Copper_River_75019800019692,75019800019692,75019800019692.00,Copper_River,47284.00,4728400.00,0.00,52.04,52.04,6.71,9.46,...,1458606400.00,163891600.00,10.10,1478801600.00,143696400.00,8.86,41098192.37,2.53,106387407.76,6.55
Copper_River_75019600118138,75019600118138,75019600118138.00,Copper_River,13745.00,1374500.00,0.00,28.09,28.09,3.29,3.94,...,3874074000.00,676138200.00,14.86,4161896100.00,388316100.00,8.53,333656068.74,7.33,5078.68,0.00
Copper_River_75019700004190,75019700004190,75019700004190.00,Copper_River,19340.00,1934000.00,0.00,36.77,36.77,9.56,7.59,...,312534100.00,57676400.00,15.58,366944100.00,3266400.00,0.88,284653.65,0.08,38076270.52,10.28
Copper_River_75019700004084,75019700004084,75019700004084.00,Copper_River,21747.00,2174700.00,0.00,37.46,37.46,6.59,6.31,...,30274900.00,3221300.00,9.62,33172200.00,324000.00,0.97,293685.88,0.88,NaN,NaN
Copper_River_75019700017692,75019700017692,75019700017692.00,Copper_River,7246.00,724600.00,0.00,24.57,24.57,3.47,3.65,...,599458200.00,100707400.00,14.38,699126400.00,1039200.00,0.15,2196422.55,0.31,403718653.00,57.64
